In [ ]:
"""
Original environment: Kaggle nb, with GPU P100 active.
Dataset (Compiled): https://www.kaggle.com/datasets/ravirvik/llm-question-classification-dataset/data
"""

# DataSources:
#     programming  -- LLMA- Copilot/code
#     https://www.kaggle.com/datasets/bhaveshmittal/python-programming-questions-dataset

#     conversation/ nlu/ basic qna   --  Text Arena
#     https://www.kaggle.com/datasets/lmsysorg/chatbot-arena-conversations

#     summarization   -- Text Arena
#     https://www.kaggle.com/datasets/pariza/bbc-news-summary

#     maths/ reasoning   --  /math
#     https://www.kaggle.com/datasets/awsaf49/math-dataset


# Fetching Data

In [ ]:
import pandas as pd
import numpy as np
import json

main_df = pd.DataFrame(columns = ["query", "class"])

coding = pd.read_csv("/kaggle/input/llm-question-classification-dataset/Python Programming Questions Dataset.csv")
# coding
temp_df = pd.DataFrame(coding["Instruction"])
temp_df["class"] = "coding"
temp_df.columns = ["query", "class"]
# temp_df
main_df = pd.concat([main_df, temp_df], ignore_index=True)
# main_df

file_path = "/kaggle/input/llm-question-classification-dataset/chatbot_arena_conversations.json"
data = []
with open(file_path, 'r') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            pass
            # print(f"Skipping line due to error: {e}")
# print(f"Loaded {len(data)} records.")
# print(data[0]["conversation_a"][0]["content"])  # Preview first record
temp = []
for i in range(len(data)):
    temp.append(data[i]["conversation_a"][0]["content"])
temp_df = pd.DataFrame(columns = ["query", "class"])
temp_df["query"] = temp
temp_df["class"] = "conversation"
main_df = pd.concat([main_df, temp_df], ignore_index=True)
# main_df

math = pd.read_csv("/kaggle/input/llm-question-classification-dataset/maths.csv")
# math
main_df = pd.concat([main_df, math], ignore_index=True)
# main_df

summary = pd.read_csv("/kaggle/input/llm-question-classification-dataset/summary.csv")
main_df = pd.concat([main_df, summary], ignore_index=True)
# main_df

main_df["class"].value_counts()
# 33k, 13k, 7.5k, 2.2k ==> 55k!!

## SMOTE:
#### To Balance different classes

In [ ]:
# smote
summary_df = main_df[main_df['class'] == 'summary']
summary_df_duplicated = pd.concat([summary_df]*2, ignore_index=True)

conversation_sample = main_df[main_df['class'] == 'conversation'].sample(n=4448, random_state=42)
coding_sample = main_df[main_df['class'] == 'coding'].sample(n=4448, random_state=42)
math_sample = main_df[main_df['class'] == 'math'].sample(n=4448, random_state=42)

mini_df = pd.concat([summary_df_duplicated, conversation_sample, coding_sample, math_sample], ignore_index=True)

mini_df = mini_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
main_df = mini_df
main_df

## Conversion to .jsonl

In [ ]:
with open('prompt_dataset.jsonl', 'w', encoding='utf-8') as f:
    for _, row in main_df.iterrows():
        entry = {
            "instruction": f"classify the following query into one of following types: conversation, code, math, summary\n ### query: {row['query']}",
            "response": row['class']
        }
        json.dump(entry, f)
        f.write('\n')

In [ ]:
data_list = []
with open('/kaggle/working/prompt_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))
print(data_list[0])

#### creating 2 samples with 100 rows each

In [ ]:
with open('/kaggle/working/prompt_dataset.jsonl', 'r', encoding='utf-8') as f:
    lines = [json.loads(line) for line in f]

first_100 = lines[:100]
second_100 = lines[100:200]

with open('sample_1.jsonl', 'w', encoding='utf-8') as f1:
    for item in first_100:
        f1.write(json.dumps(item) + '\n')

with open('sample_2.jsonl', 'w', encoding='utf-8') as f2:
    for item in second_100:
        f2.write(json.dumps(item) + '\n')